In [1]:
import gymnasium as gym
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
import matplotlib.pyplot as plt
from gymnasium.wrappers import RecordVideo
import torch
import torch.nn as nn
%matplotlib inline

2024-01-17 15:06:47.282166: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-17 15:06:47.305413: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 15:06:47.305442: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 15:06:47.306078: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-17 15:06:47.310471: I tensorflow/core/platform/cpu_feature_guar

In [3]:
def make_env(env_id):
    def _init():
        env = gym.make(env_id, max_episode_steps=800, continuous=False)
        env.reset()
        return env
    return _init

In [4]:
n_procs = 16
env_id = "CarRacing-v2"
TRAIN_STEPS = 2000000
EVAL_EPS = 20

In [5]:
train_env = SubprocVecEnv([make_env(env_id) for _ in range(n_procs)],start_method="fork")
eval_env = gym.make(env_id, max_episode_steps=3200, continuous=False)

In [10]:
train_env.reset()
model = DQN("CnnPolicy", train_env, verbose=1, buffer_size=100000, tensorboard_log='tensorboard_logs/')
model.learn(total_timesteps=TRAIN_STEPS, tb_log_name="first_run")
mean_reward, _ = evaluate_policy(model, eval_env, n_eval_episodes=EVAL_EPS)
train_env.close()
print(mean_reward)
model.save("dqn_carracing")

Using cuda device
Wrapping the env in a VecTransposeImage.
Logging to tensorboard_logs/first_run_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.939    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 949      |
|    time_elapsed     | 13       |
|    total_timesteps  | 12800    |
----------------------------------
------------------------------
| time/              |       |
|    episodes        | 8     |
|    fps             | 949   |
|    time_elapsed    | 13    |
|    total_timesteps | 12800 |
------------------------------
------------------------------
| time/              |       |
|    episodes        | 12    |
|    fps             | 949   |
|    time_elapsed    | 13    |
|    total_timesteps | 12800 |
------------------------------
------------------------------
| time/              |       |
|    episodes        | 16    |
|    fps             | 949   |
|    time_elapsed    | 13   

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


712.7648527156562


In [2]:
model = DQN.load("dqn_carracing")

In [5]:
eval_env = gym.make(env_id, max_episode_steps=3200, continuous=False, render_mode="rgb_array")
eval_env = RecordVideo(eval_env, 'video')
eval_env = DummyVecEnv([lambda: eval_env])

/usr/local/lib/python3.11/dist-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /tf/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [6]:
mean_reward, _ = evaluate_policy(model, eval_env, n_eval_episodes=1)

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Moviepy - Building video /tf/video/rl-video-episode-0.mp4.
Moviepy - Writing video /tf/video/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /tf/video/rl-video-episode-0.mp4
